# Code to connect to MySQL instance


In [2]:
!pip install mysql-connector-python
import mysql.connector

Defaulting to user installation because normal site-packages is not writeable


## Ensure that iP address is allowed by google coud to connect to the instance.

In [10]:
import mysql.connector

config = {
    'user': 'james',
    'password': '******',
    'host': '34.147.246.76', # public IP of the VM
    'database': 'LostAndFound',
    'port': 3306,
    'raise_on_warnings': True
}

try:
    conn = mysql.connector.connect(**config)
    cursor = conn.cursor()
    print("Connected!")
except Exception as e:
    print("Connection error:", e)


Connected!


### Tables in the DB

In [14]:
cursor.execute("SHOW TABLES")
tables = cursor.fetchall()
print("Tables in database:")
for table in tables:
    print(table[0])

Tables in database:
Claims
Lost_Items
Users


## Claims

In [20]:
table_name = 'Claims'
cursor.execute(f"DESCRIBE {table_name}")
columns = cursor.fetchall()

print(f"Columns in `{table_name}`:")
for col in columns:
    print(col[0], '-', col[1])

Columns in `Claims`:
Claim_ID - int
User_ID - int
Item_ID - int
Status - enum('Pending','Verified','Rejected')
Submitted_At - timestamp
Verified_By - int
Resolution_Date - timestamp


## Lost_Items

In [21]:
table_name = 'Lost_Items'
cursor.execute(f"DESCRIBE {table_name}")
columns = cursor.fetchall()

print(f"Columns in `{table_name}`:")
for col in columns:
    print(col[0], '-', col[1])

Columns in `Lost_Items`:
Item_ID - int
Name - varchar(100)
Category - varchar(50)
Location - varchar(100)
Found_Date - date
Description - text
Status - enum('Lost','Claimed','Returned')
User_ID - int


## Users

In [22]:
table_name = 'Users'
cursor.execute(f"DESCRIBE {table_name}")
columns = cursor.fetchall()

print(f"Columns in `{table_name}`:")
for col in columns:
    print(col[0], '-', col[1])

Columns in `Users`:
User_ID - int
Name - varchar(100)
Email - varchar(255)
Password_Hash - varchar(255)
Role - enum('Admin','User','Employee')
Created_At - timestamp


## Now for Actual CRUD Operations

*  Use faker for data generation.

In [25]:
from faker import Faker
import random
from datetime import datetime, timedelta

fake = Faker()

## Generate and Insert Users

In [26]:
user_data = []

for i in range(1, 51):
    name = fake.name()
    email = fake.email()
    password_hash = fake.sha256()
    role = random.choice(["User", "Employee", "Admin"])
    created_at = fake.date_time_between(start_date="-1y", end_date="now").strftime("%Y-%m-%d %H:%M:%S")
    user_data.append((i, name, email, password_hash, role, created_at))

insert_users = """
INSERT INTO Users (User_ID, Name, Email, Password_Hash, Role, Created_At)
VALUES (%s, %s, %s, %s, %s, %s)
"""

cursor.executemany(insert_users, user_data)
conn.commit()
print(f"Inserted {cursor.rowcount} dummy users")


Inserted 50 dummy users


## Generate and Insert Lost_Items

In [28]:
item_data = []

categories = ["Electronics", "Clothing", "Accessories", "Books", "Keys", "ID Cards"]
locations = ["Library", "Cafeteria", "Gym", "Auditorium", "Parking Lot", "Restroom"]
statuses = ["Lost", "Claimed", "Returned"]

for i in range(1, 51):
    name = fake.word().capitalize()
    category = random.choice(categories)
    location = random.choice(locations)
    found_date = fake.date_between(start_date="-6M", end_date="today")
    description = fake.sentence(nb_words=12)
    status = random.choice(statuses)
    user_id = random.randint(1, 50)  # Assuming users with IDs 1-50 exist

    item_data.append((i, name, category, location, found_date, description, status, user_id))

insert_items = """
INSERT INTO Lost_Items (
    Item_ID, Name, Category, Location, Found_Date, Description, Status, User_ID
)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
"""

cursor.executemany(insert_items, item_data)
conn.commit()
print(f"Inserted {cursor.rowcount} corrected dummy lost items")


Inserted 50 corrected dummy lost items


## Insert Claims data

In [ ]:
claim_data = []

statuses = ["Pending", "Verified", "Rejected"]

for i in range(1, 51):
    user_id = random.randint(1, 50)
    item_id = random.randint(1, 50)
    status = random.choice(statuses)
    submitted_at = fake.date_time_between(start_date="-3M", end_date="now").strftime("%Y-%m-%d %H:%M:%S")
    verified_by = random.randint(1, 50) if status != "Pending" else None
    resolution_date = (
        (datetime.strptime(submitted_at, "%Y-%m-%d %H:%M:%S") + timedelta(days=random.randint(1, 10)))
        if status != "Pending" else None
    )

    claim_data.append((i, user_id, item_id, status, submitted_at, verified_by, resolution_date))

insert_claims = """
INSERT INTO Claims (
    Claim_ID, User_ID, Item_ID, Status, Submitted_At, Verified_By, Resolution_Date
)
VALUES (%s, %s, %s, %s, %s, %s, %s)
"""

cursor.executemany(insert_claims, claim_data)
conn.commit()
print(f"Inserted {cursor.rowcount} dummy claims")

In [31]:
import pandas as pd

df_users = pd.read_sql("SELECT * FROM Users", conn)
df_users.head()

/var/folders/mr/_57ysvt11t954qfbkzmprqqm0000gn/T/ipykernel_15096/3921522526.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_users = pd.read_sql("SELECT * FROM Users", conn)


,User_ID,Name,Email,Password_Hash,Role,Created_At
0,1,Travis Zuniga,jfletcher@example.net,48dee48ee7c4c782da678a50d3445780e41b1e24cac8b6...,Employee,2024-09-14 23:07:52
1,2,Jim Young,whiteheadlindsay@example.org,221c291335796cd8c30088305f4fa711638dd668a045b1...,Admin,2025-03-29 14:09:49
2,3,Joel Gonzalez,whitechristopher@example.com,0646ea27b1c753f71813988502ffb6484ed1e71dd83dc9...,User,2025-01-19 14:20:55
3,4,James Travis,jodisanchez@example.net,c8b40782d0580d578f565c2049cca455317f305c8aa6c0...,Employee,2024-07-23 01:42:55
4,5,Tiffany Sanchez,sarah82@example.org,b585ef9e943701f734981f634596f36f8ccbfe9fcd83ca...,Employee,2024-07-11 12:05:15


In [32]:
df_items = pd.read_sql("SELECT * FROM Lost_Items", conn)
df_items.head()

/var/folders/mr/_57ysvt11t954qfbkzmprqqm0000gn/T/ipykernel_15096/4081135041.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_items = pd.read_sql("SELECT * FROM Lost_Items", conn)


,Item_ID,Name,Category,Location,Found_Date,Description,Status,User_ID
0,1,Blue,Accessories,Auditorium,2024-09-29,Discover power fire machine society network fi...,Returned,50
1,2,Wish,Keys,Cafeteria,2025-03-11,Seven inside direction final by more political.,Claimed,37
2,3,Chair,Keys,Parking Lot,2025-02-24,Perhaps country continue weight off perform wo...,Returned,38
3,4,Capital,Books,Parking Lot,2024-10-21,Degree dinner until than degree current she in...,Returned,32
4,5,Seven,Accessories,Library,2025-01-31,Special subject effort job rather something we...,Lost,45


In [33]:
df_claims = pd.read_sql("SELECT * FROM Claims", conn)
df_claims.head()

/var/folders/mr/_57ysvt11t954qfbkzmprqqm0000gn/T/ipykernel_15096/3962198764.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_claims = pd.read_sql("SELECT * FROM Claims", conn)


,Claim_ID,User_ID,Item_ID,Status,Submitted_At,Verified_By,Resolution_Date
0,1,35,14,Rejected,2025-02-27 13:27:54,45.0,2025-03-04 13:27:54
1,2,40,13,Pending,2025-02-13 08:52:37,NaN,NaT
2,3,4,15,Rejected,2025-02-08 14:25:22,2.0,2025-02-18 14:25:22
3,4,18,6,Rejected,2025-02-11 07:40:43,22.0,2025-02-20 07:40:43
4,5,14,48,Pending,2025-01-22 11:47:59,NaN,NaT


In [34]:
df_lost = pd.read_sql("SELECT * FROM Lost_Items WHERE Status = 'Lost'", conn)
df_lost.head()

/var/folders/mr/_57ysvt11t954qfbkzmprqqm0000gn/T/ipykernel_15096/2006005169.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_lost = pd.read_sql("SELECT * FROM Lost_Items WHERE Status = 'Lost'", conn)


,Item_ID,Name,Category,Location,Found_Date,Description,Status,User_ID
0,5,Seven,Accessories,Library,2025-01-31,Special subject effort job rather something we...,Lost,45
1,6,Themselves,Accessories,Gym,2025-01-18,Floor make leader culture good week spring gun...,Lost,21
2,9,Degree,Books,Library,2024-12-24,End no home open her across very kid.,Lost,49
3,13,Long,Electronics,Auditorium,2024-12-30,Treat speak rest whole prove world really late...,Lost,7
4,14,Arm,Books,Restroom,2024-12-08,Able reason future system and create seek exac...,Lost,12
